In [9]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pandas as pd
import re

class PreprocessingText:
    def _init_(self):
        self.slang_list = [kamus.strip('\n').strip('\r') for kamus in open('/slang.txt')]
        self.stopword_list = [line.strip('\n')for line in open('/stop.txt')]
        
    # Case Folding
    def CaseFolding(self, tweets):
        tweets = tweets.lower()
        
        return tweets
    
    # Cleansing
    def Cleansing(self, tweets):
        # Menghilangkan mention, hashtag, character reference
        tweets = re.sub('[@#&][A-Za-z0-9_]+'," ", tweets)

        # Menghilangkan Tautan
        tweets = re.sub("\w+:\/\/\S+"," ", tweets)

        # Menghilangkan tanda baca
        tweets = re.sub('[()!?;,]', ' ', tweets)
        tweets = re.sub('\[.*?\]',' ', tweets)

        # Menghilangkan tanda selain huruf
        tweets = re.sub("[^a-z]"," ", tweets)

        # Menghilangkan spasi lebih dari 1
        tweets = ' '.join(tweets.split())
        
        return tweets
    
    # Replace Slangwords
    def Slangwords(self, tweets):
        for line in self.slang_list:
            slangword = line.strip().split(":")
            if slangword[0] in tweets.split(" "):
                tweets = tweets.replace(slangword[0] + " ", slangword[1] + " ")
                tweets = tweets.replace(" " + slangword[0], " " + slangword[1])
                
        return tweets
    
    # Stopwords Removal
    def StopwordsRemoval(self, tweets):
        stopwords_list = list(map(lambda x:x.strip(),list(self.stopword_list)))
        tweets = tweets.split()
        tweets = [w for w in tweets if not w in stopwords_list]
        tweets = " ".join(word for word in tweets)
        
        return tweets
    
    # Tokenization
    def Tokenization(self, tweets):
        tweets = tweets.split()
        
        return tweets
    
    # Normalization (Stemming)
    def Stemming(self, tweets):
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()
        do = []
        for w in tweets:
            dt = stemmer.stem(w)
            do.append(dt)
        d_clean = []
        d_clean = " ".join(do)
        print(d_clean)
        
        return d_clean
    
    def Stemming1(self, tweets):
        stemmer = StemmerFactory.create_stemmer()
        tweets = stemmer.stem(tweets)
        print(tweets)

        return tweets

In [10]:
df = pd.read_csv("Final_Sebelum_Kanjuruhan.csv", sep=';', encoding='unicode_escape')
df

,id,conversation_id,created_at,user_id,username,label,tweet
0,"1,56E+18","1,56E+18",2022-08-31 06:58:15 SE Asia Standard Time,"1,43E+18",pinem3paska,NEGATIF,@Indostransfer Harapan saya Agar Di Sumatera ...
1,"1,56E+18","1,56E+18",2022-08-31 06:54:57 SE Asia Standard Time,968410669,bhaniie_bsf,NEGATIF,@ocehangaluh Haha gimana mau degradasi orang d...
2,"1,56E+18","1,56E+18",2022-08-31 06:53:15 SE Asia Standard Time,116308596,awaveiro,NEGATIF,"Detail banget woy, sampe pager betis lompat aj..."
3,"1,56E+18","1,56E+18",2022-08-31 06:48:16 SE Asia Standard Time,"1,45E+18",loyalisgaruda,NEGATIF,@abdulra55101674 @sahaaat_ @Vand0e @OrenDepok ...
4,"1,56E+18","1,56E+18",2022-08-31 06:45:57 SE Asia Standard Time,124392857,rahmaddiant0,NEGATIF,@PSSI Bukan sobat ambyar?
...,...,...,...,...,...,...,...
539,"1,48E+18","1,48E+18",2022-01-01 06:02:38 SE Asia Standard Time,"1,12E+18",fachsetiawan,POSITIF,@lisadepe Ga salah dong gw ngidolain Ajax keti...
540,"1,48E+18","1,48E+18",2022-01-01 06:02:38 SE Asia Standard Time,"7,08E+17",dulqowi,POSITIF,"@fedriza @PSSI Idem bro, sementara ini dulu aj..."
541,"1,48E+18","1,48E+18",2022-01-01 06:02:04 SE Asia Standard Time,"1,28E+18",ganesha010180,NEGATIF,"@SiaranBolaLive Ga habis pikir, PSSI menggunak..."
542,"1,48E+18","1,48E+18",2022-01-01 05:59:26 SE Asia Standard Time,"1,33E+18",indones30065690,NEGATIF,@tirta_cipeng @PSSI Perusak Sepakbola Sejati I...


In [11]:
%%time
pretext = PreprocessingText()

# ======================================================================================================================

df_casefolding = df.copy()
df_casefolding['tweet'] = df_casefolding['tweet'].apply(pretext.CaseFolding)

df_cleansing = df_casefolding.copy()
df_cleansing['tweet'] = df_cleansing['tweet'].apply(pretext.Cleansing)
# df_cleansing = df_cleansing.drop_duplicates(subset='tweet')

df_slangwords = df_cleansing.copy()
df_slangwords['tweet'] = df_slangwords['tweet'].apply(pretext.Slangwords)

df_stopwords = df_slangwords.copy()
df_stopwords['tweet'] = df_stopwords['tweet'].apply(pretext.StopwordsRemoval)

df_token = df_stopwords.copy()
df_token['tweet'] = df_token['tweet'].apply(pretext.Tokenization)

df_stemming = df_token.copy()
# df_stemming['tweet'] = df_stemming['tweet'].apply(pretext.Stemming)

dataAll = {'Raw Data': df, 'Case Folding': df_casefolding, 'Cleansing': df_cleansing, 
           'Normalization': df_slangwords, 'StopWords Removal': df_stopwords, 'Stemming': df_stemming,
           'Tokenization': df_token}

writer = pd.ExcelWriter('./kanjuruhan_sesudah_Preprocessed.xlsx', engine='xlsxwriter', mode='w')

for data_sheet in dataAll.keys():
    dataAll[data_sheet].to_excel(writer, sheet_name=data_sheet, index=False)

writer.close()
writer.handles = None

print("\n\n\nData Preprocessed Saved!!!")

AttributeError: 'PreprocessingText' object has no attribute 'slang_list'

In [81]:
import pandas as pd

data = pd.read_csv("Final_Sesudah_Kanjuruhan.csv", sep=';', encoding='unicode_escape')
# data = pd.read_csv("Final_Sesudah_Kanjuruhan.csv", sep=';', encoding='unicode_escape')
data

,id,conversation_id,created_at,user_id,username,label,tweet
0,"1,58E+18","1,58E+18",2022-10-16 05:18:19 SE Asia Standard Time,"1,49E+18",alhusaini_asror,NEGATIF,"@mohmahfudmd Benar dan Betul.Ketua PSSI,mundur..."
1,"1,58E+18","1,58E+18",2022-10-16 05:13:51 SE Asia Standard Time,1442933096,metmalamminggu,NEGATIF,"@iIhamzada Dilarang intervensi gara"" aturan fi..."
2,"1,58E+18","1,58E+18",2022-10-16 05:06:34 SE Asia Standard Time,"7,88E+17",onedayasmine,NEGATIF,@gibran_tweet Mas.. jadi ketua PSSI yaaa Kalau...
3,"1,58E+18","1,58E+18",2022-10-16 05:03:44 SE Asia Standard Time,4812772283,harryunited05,NEGATIF,@medioclubID @PSSI mana orangnya itu2 aja lagi...
4,"1,58E+18","1,58E+18",2022-10-16 05:00:24 SE Asia Standard Time,"1,31E+18",ariflabmed,POSITIF,@medioclubID @PSSI Yunus Nusi akan dikenang se...
...,...,...,...,...,...,...,...
1273,"1,58E+18","1,58E+18",2022-10-07 11:47:59 SE Asia Standard Time,521254614,bodoamath,NEGATIF,lho kapolda sama ketua pssi nya mana?
1274,"1,58E+18","1,58E+18",2022-10-07 11:47:24 SE Asia Standard Time,"1,57E+18",crypto34209511,NEGATIF,@PSSI HAK DAN KEWAJIBAN PENONTON HARUS DI INGA...
1275,"1,58E+18","1,58E+18",2022-10-07 11:43:10 SE Asia Standard Time,2156012929,gbrand_9127,NEGATIF,@CNNIndonesia PSSI juga gk standar FIFA.
1276,"1,58E+18","1,58E+18",2022-10-07 11:41:33 SE Asia Standard Time,480858414,grrraargh,NEGATIF,@ainurohman Dia pikir PSSI cuma urus timnas


In [82]:
tweets_positive = [t for t in data["label"] if t == "POSITIF"]
tweets_negative = [t for t in data["label"] if t == "NEGATIF"]

print("Hasil Sentimen Analisis")
print("POSITIF : ", len(tweets_positive), "({}%)".format(100*len(tweets_positive)/len(data)))
print("NEGATIF : ", len(tweets_negative), "({}%)".format(100*len(tweets_negative)/len(data)))
print("TOTAL : ", len(data))

Hasil Sentimen Analisis
POSITIF :  639 (50.0%)
NEGATIF :  639 (50.0%)
TOTAL :  1278
